In [1]:
import rioxarray as rioxr
import rasterio
import geopandas as gpd
import os

from shapely.geometry import box

In [2]:
year = 2012

In [3]:
# file paths to rasters
root = os.path.join(os.getcwd(),
                        'S_clip_preds',
                        'S_clip_preds_'+str(year))

s_fp = os.path.join(root, 'merged_crs26910_S_' + str(year) + '.tif')
w_fp = os.path.join(root, 'merged_crs26910_W_' + str(year) + '.tif') 
r_fp = os.path.join(root, 'merged_crs26911_' + str(year) + '.tif')

In [4]:
# open crs26910 south raster and bounding box
s_raster = rioxr.open_rasterio(s_fp).squeeze()
s_bbox = box(*s_raster.rio.bounds())

# ------------- CLIP crs26910 W raster ----------
w_raster = rioxr.open_rasterio(w_fp).squeeze()
w_bbox = box(*w_raster.rio.bounds())

if (s_bbox.intersects(w_bbox)):
    new_bbox = w_bbox.difference(s_bbox)
    clipped_raster = w_raster.rio.clip([new_bbox]).squeeze()

    # save clipped raster
    with rasterio.open(
                os.path.join(root, 'merged_crs26910_W_' + str(year) + '_clip.tif'),  # file path
                'w',           # w = write
                driver = 'GTiff', # format
                height = clipped_raster.shape[0], 
                width = clipped_raster.shape[1],
                count = 1,  # number of raster bands in the dataset
                dtype = rasterio.uint8,
                crs = clipped_raster.rio.crs,
                transform = clipped_raster.rio.transform(),
            ) as dst:
                dst.write(clipped_raster.astype(rasterio.uint8), 1)
    os.remove(w_fp)

In [5]:
# ------------- CLIP crs26911 raster ----------
# open crs26911 raster and bouding box
r_raster = rioxr.open_rasterio(r_fp).squeeze()
r_bbox = box(*r_raster.rio.bounds())

# match crs of bounding boxes
s_gdf = gpd.GeoDataFrame({'geometry': [s_bbox]}, crs= s_raster.rio.crs )
s_gdf = s_gdf.to_crs(r_raster.rio.crs)
s_bbox = s_gdf.geometry[0]

if (s_bbox.intersects(r_bbox)):
    # take difference of r_bbox and s_bbox
    new_bbox = r_bbox.difference(s_bbox)

    # clip crs26911 raster
    clipped_raster = r_raster.rio.clip([new_bbox]).squeeze()

    # save clipped raster
    with rasterio.open(
                os.path.join(root, 'merged_crs26911_' + str(year) + '_clip.tif'),  # file path
                'w',           # w = write
                driver = 'GTiff', # format
                height = clipped_raster.shape[0], 
                width = clipped_raster.shape[1],
                count = 1,  # number of raster bands in the dataset
                dtype = rasterio.uint8,
                crs = clipped_raster.rio.crs,
                transform = clipped_raster.rio.transform(),
            ) as dst:
                dst.write(clipped_raster.astype(rasterio.uint8), 1)
    os.remove(r_fp)